Anmerkung: Im folgenden Skript gibt es keine Ausgaben, da ich das Modell mithilfe der im GPU-Labor zur Verfügung stehenden Rechner trainiert habe. Ich habe die unten aufgeführten Befehle in das Terminal eingegeben, dass ich es dort im Hintergrund trainieren lassen konnte.

Anmerkung Datensatz:
Der Datensatz liegt in folgender Struktur vor:
- annotations/
    - instances_train.json   &emsp;&emsp; Informationen im COCO-Format über die Trainingsbilder
    - instances_val.json  &emsp;&emsp; Informationen im COCO-Format über die Validationbilder
- train2017/ Trainingsbilder
- val2017/ Validationbilder

# YoloX Training

basiert auf dem YoloX GitHub Repository von Megvii-BaseDetection (https://github.com/Megvii-BaseDetection/YOLOX)

**Ordnerstruktur:**

yoloxModel  
├── datasets                &emsp;&emsp; Datensatz im COCO-Format
├── testImages              &emsp;&emsp; Testbilder für demo.py  
├── weights                 &emsp;&emsp; Pre-Trained Gewichte aus dem Repository (Pre-Trained mit COCO-Datensatz)  
├── yolox                   &emsp;&emsp; YoloX-Netzwerk  
├── YOLOX_Outputs           &emsp; Ausgabe der Gewichte und Log-Dateien nach dem Training und der Vorhersage  
├── requirements.txt        &emsp;&emsp; Requirements für die Installation der Pakete  
├── train.py                &emsp;&emsp; Datei zum Trainieren  
├── eval.py                 &emsp;&emsp; Datei zum Evaluieren  
├── demo.py                 &emsp;&emsp; Datei zur Vorhersage  
└── yolox_m.py              &emsp;&emsp; Konfigurationsdatei (hier für yolox in Größe m)  
--> Andere Konfigurationsdateien für YoloX in einer anderen Größe können in dem GitHub Repository von Megvii-BaseDetection unter '/exps/default/' gefunden werden

Die folgenden Skripte sind aus dem YoloX GitHub Repository
- train.py Training des YoloX-Netzwerks
- eval.py Validation des YoloX-Netzwerks
- demo.py Vorhersage mit YoloX-Netzwerk
- yolox_m.py Dort sind die Konfigurationen für das Yolo-Netzwerk in Größe Medium (m). Hyperparemeter werden dort konfiguriert und Pfade zum Datensatz angepasst

## Vor dem Training


Vor dem Training muss die requirements.txt installiert werden.

In [ ]:
!pip install -r requirements.txt

Passe die yolox_m.py (Konfigurationsdatei) an:

Bei dieser Datei handelt es sich um eine Klasse, die alle Hyperparameter beinhaltet, die für das YoloX-Netzwerk benötigt werden.
Alle Default-Parameter können in folgedner Datei 'yolox/exp/yolox_base,p' eingesehen werden und müssen, falls nötig in der yolox_m.py überschrieben werden, da sonst diese Werte genommen werden.

- self.depth = 0.67
- self.width = 0.75
- self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]  
--> Parameter für Modell (ob nano, s, m, l, ...)  

Meine angepassten Parameter:
- self.data_dir = "datasets/dataYoloX"   &emsp;&emsp; # Ordner in dem die Datensatzordner annotation, train2017 und val2017 liegen
- self.train_ann = "instances_train.json"  &emsp; # .json Datei mit den Informationen über die Trainingsdaten
- self.val_ann = "instances_val.json" &emsp; # .json Datei mit den Informationen über die Validationdaten
- self.num_classes = 5  &emsp;&emsp; # Anzahl der Klassen in dem Datensatz
- self.max_epoch = 100  &emsp;&emsp; # Anzahl von Epochen
- self.data_num_workers = 4  &emsp;&emsp; # Anzahl der Prozesse
- self.eval_interval = 1  &emsp;&emsp; # Anzahl wie oft das netzwerk in dem Training evaluiert wird


## Training

Beim Training werden die Gewichte der aktuellen Epoche, die besten Gewichte (best-ckpt.pth), das Trainings-Log (train_log.txt) und die Trainingsmetriken mithilfe von tensorboard gespeichert. Während des Trainings werden die Dateien unter dem Pfad 'YOLOX_outputs/yolox_m/' gespeichert.

Aufgrund der Größe der Dateien sind hier nicht alle Gewichte abgespeichert, sondern nur die Letzten und die Besten.

Für das Training mit Oversampling wurde der torch.utils.data.WeightedRandomSampler() von PyTorch genommen, dieser hat dann im code 'yolox/data/dataloading.py' den RandomSampler() ersetzt.<br>
Die Ergebnisse mit den Default-Werten sind in 'YOLOX_outputs/yolox_m_100_epoch_no_Oversample'.<br>
Die Ergebnisse mit den WeightedRandomSampler sind in 'YOLOX_outputs/yolox_m_100_epoch_Oversample'.

Parameter erklären:
- -f Pfad zur Konfigurationsdatei
- -d Anzahl der GPUs
- -b Batchsize
- -o Benutze GPU RAM
- -c Pfad zu den  Pre-Trained Gewichten


In [ ]:
# zum ausführen des Trainings
!python train.py -f yolox_m.py -d 1 -b 8 --fp16 -o -c weights/yolox_m.pth

## Validation

Parameter erklären:
- -n Yolo Modell (ob s: yolox-s oder m: yolox-m, ...)
- -c Pfad zu den trainierten Modellgewichten
- -d Anzahl der GPUs
- -b Batchsize
- --conf Threshold
- -f Pfad zur Konfigurationsdatei

In [ ]:
!python3 eval.py -n  yolox-m -c YOLOX_outputs/yolox_m_100_epoch_no_Oversample/weights/best_ckpt.pth -b 8 -d 1 --conf 0.001 -f yolox_m.py

In [ ]:
!python3 eval.py -n  yolox-m -c YOLOX_outputs/yolox_m_100_epoch_Oversample/weights/best_ckpt.pth -b 8 -d 1 --conf 0.001 -f yolox_m.py

## Prediction
Trifft eine Vorhersage auf ein angegebenes Bild. Das Ergebnis wird unter 'YOLOX_outputs/<yolox_model>/vis_res' abgespeichert, falls --save_result gesetzt ist.

Parameter erklären:
- -f Pfad zur Konfigurationsdatei
- -c Pfad zu den Gewichten
- --path Pfad zu dem Testfoto (.jpg, .png)
- --conf Threshold
- --nms Threshold (Non-max Suppresion)
- --tsize Dimension des Testbildes (Bsp.: 512x512)
- --save_result Das Ergebnis wird unter 'YOLOX_outputs/yolox_m/vis_res/' gespeichert
- -- device Ob CPU oder GPU

In [15]:
!python3 demo.py image -f yolox_m.py -c YOLOX_outputs/yolox_m_100_epoch_no_Oversampling/weights/best_ckpt.pth --path testImages/testImage_1.jpg --conf 0.25 --nms 0.45 --tsize=512 --save_result --device cpu

2023-06-30 12:48:46.566 | INFO     | __main__:main:259 - Args: Namespace(demo='image', experiment_name='yolox_m', name=None, path='testImages/testImage_3.jpg', camid=0, save_result=True, exp_file='yolox_m.py', ckpt='YOLOX_outputs/yolox_m_100_epoch_no_Oversampling/weights/best_ckpt.pth', device='cpu', conf=0.25, nms=0.45, tsize=512, fp16=False, legacy=False, fuse=False, trt=False)
2023-06-30 12:48:47.249 | INFO     | __main__:main:269 - Model Summary: Params: 25.28M, Gflops: 47.19
2023-06-30 12:48:47.250 | INFO     | __main__:main:282 - loading checkpoint
2023-06-30 12:48:50.785 | INFO     | __main__:main:286 - loaded checkpoint done.
2023-06-30 12:48:51.607 | INFO     | __main__:inference:165 - Infer time: 0.8148s
2023-06-30 12:48:51.608 | INFO     | __main__:image_demo:202 - Saving detection result in ./YOLOX_outputs/yolox_m/vis_res/2023_06_30_12_48_50/testImage_3.jpg
^C


In [ ]:
!python3 demo.py image -f yolox_m.py -c YOLOX_outputs/yolox_m_100_epoch_Oversampling/weights/best_ckpt.pth --path ../testImage/testImage_1.png --conf 0.25 --nms 0.45 --tsize=512 --save_result --device cpu